In [3]:
import numpy as np
import pandas as pd
import rasterio as r
import os

In [15]:
# import the data
df=pd.read_csv('archive.csv').dropna( thresh=3 )

# remove the last two columns

df = df.iloc[:, :-2]

# set the first column as interger type

df.iloc[:, 0] = df.iloc[:, 0].astype(int)

# drop the datas with year less than 2000

df = df[df.iloc[:, 0] >= 2000]

#check
df

,Year,Name,Diameter,Latitude,Longitude,Start_Date,End_Date
8,2000.0,Slyudyanka,5.6,103.83,51.68,04/27,04/27
9,2000.0,Severobaykalsk,5.4,109.37,55.35,05/15,05/15
10,2001.0,Krestovskiy C.,4.4,106.34,52.55,04/21,05/10
11,2002.0,M.More North,7.6,107.70,53.46,04/19,04/26
12,2002.0,M.More South,3.4,107.14,53.24,04/19,04/26
13,2002.0,Olkhon East,7.6,107.58,53.09,04/26,04/26
14,2003.0,Krestovskiy C.,5.2,106.45,52.60,04/03,04/28
15,2003.0,Off Krestovskiy C.,4.4,106.81,52.58,04/17,05/08
16,2003.0,Hovsgol,5.8,100.42,51.04,06/13,06/13
17,2004.0,Krestovskiy C.,6.0,106.42,52.59,04/21,05/02


In [7]:
# define a gee code generator function
def gee_code_generator(year,lat,lon,start_date,end_date):
    start_date = str(start_date).split('/')
    start_date = str(year) + '-' + start_date[0] + '-' + start_date[1]

    end_date = str(end_date).split('/')
    end_date = str(year) + '-' + end_date[0] + '-' + end_date[1]
    
    return """
var dataset = ee.ImageCollection('MODIS/061/MOD09A1')
                  .filter(ee.Filter.date('{}', '{}'));
var trueColor =
    dataset
    .first().clip(Baikal);
var trueColorVis = {{
  min: -100.0,
  max: 8000.0,
  bands:['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03']
}};
Map.setCenter({},{})
Map.addLayer(trueColor, trueColorVis, 'True Color');

var selectedBands = trueColor.select(['sur_refl_b01', 'sur_refl_b02','sur_refl_b03','sur_refl_b04'
,'sur_refl_b05','sur_refl_b06','sur_refl_b07']);

var exportParameters = {{
  image: selectedBands,
  description: 'Baikal_14_04_15',
  scale: 50,
  region: Baikal, // make sure this is defined correctly
  fileFormat: 'GeoTIFF',
  crs: 'EPSG:4326' ,
  maxPixels: 1e9
}};

Export.image.toDrive(exportParameters);""".format(start_date,end_date,lat,lon)
    

code = gee_code_generator(2000, 53.558, 107.337, '14/04', '15/04')
print(code)


var dataset = ee.ImageCollection('MODIS/061/MOD09A1')
                  .filter(ee.Filter.date('2000-14-04', '2000-15-04'));
var trueColor =
    dataset
    .first().clip(Baikal);
var trueColorVis = {
  min: -100.0,
  max: 8000.0,
  bands:['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03']
};
Map.setCenter(53.558,107.337)
Map.addLayer(trueColor, trueColorVis, 'True Color');

var selectedBands = trueColor.select(['sur_refl_b01', 'sur_refl_b02','sur_refl_b03','sur_refl_b04'
,'sur_refl_b05','sur_refl_b06','sur_refl_b07']);

var exportParameters = {
  image: selectedBands,
  description: 'Baikal_14_04_15',
  scale: 50,
  region: Baikal, // make sure this is defined correctly
  fileFormat: 'GeoTIFF',
  crs: 'EPSG:4326' ,
  maxPixels: 1e9
};

Export.image.toDrive(exportParameters);


In [18]:
# define a text_to_xml function

def text_to_xml(t=input()):
    """text_to_xml function converts a string to an xml file

    Args:
        t (string, optional): a series of coordinates in inproper formate. Defaults to input().
        for expample: 
        ee.Geometry.Polygon(
        [[[108.2831038892732, 53.545549728843156],
          [108.2831038892732, 53.44342842105715],
          [108.46643823985914, 53.44342842105715],
          [108.46643823985914, 53.545549728843156]]], null, false);

    """
    
    coordinates = t.split('[')
    coordinates = coordinates[3:]
    coordinates = [i.split(']')[0] for i in coordinates]
    coordinates = [i.split(', ') for i in coordinates]
    coordinates = [[float(j) for j in i] for i in coordinates]
    
    xml = f"""
<Polygon>
    <outerBoundaryIs>
        <LinearRing>
            <coordinates>
            {' '.join([f'{i[0]},{i[1]}' for i in coordinates])}
            </coordinates>
        </LinearRing>
    </outerBoundaryIs>
</Polygon>"""
    
    return xml

print(text_to_xml(t='''ee.Geometry.Polygon(
        [[[108.2831038892732, 53.545549728843156],
          [108.2831038892732, 53.44342842105715],
          [108.46643823985914, 53.44342842105715],
          [108.46643823985914, 53.545549728843156]]], null, false);
'''))



<Polygon>
    <outerBoundaryIs>
        <LinearRing>
            <coordinates>
            108.2831038892732,53.545549728843156 108.2831038892732,53.44342842105715 108.46643823985914,53.44342842105715 108.46643823985914,53.545549728843156
            </coordinates>
        </LinearRing>
    </outerBoundaryIs>
</Polygon>
